# Master 2023 - Degradation of Norwegian Wind Farms

To run this program, all the files in Data/Værdata at OneDrive need to be uploaded

In [ ]:
#Installing required libraries
!pip install windpowerlib
!pip install windrose
!pip install pmdarima
!pip install xarray==0.21.1
!pip install geopy

In [ ]:
# Libraries for working with multidimensional arrays
import numpy as np
import xarray as xr
# Libraries for plotting and visualising data
import matplotlib.path as mpath
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import altair as alt
from windrose import WindroseAxes
import seaborn as sns

from windpowerlib import ModelChain, WindTurbine, create_power_curve
from windpowerlib import data as wt
from windpowerlib.power_output import power_coefficient_curve

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


import statsmodels.api as sm
import statsmodels.stats.diagnostic as smd
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from statsmodels.stats.diagnostic import linear_rainbow,linear_harvey_collier,het_breuschpagan

import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.regression.linear_model import OLS
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.seasonal import seasonal_decompose

import statsmodels.graphics.tsaplots as tsa
from scipy import stats

import geopy.distance



# Functions for treating production data

In [ ]:
power_coeff=pd.read_csv("https://pvexpect.com/Green_hydrogen/power_coefficient_2.csv", delimiter=";")

def create_production_dataframe(chosen_windfarms):

  # we start by loading the wind energy production data from an Excel file.
  wind_energy = pd.read_excel("NVE_prod_2002_2023.xlsx")

  #@title Data structuring of wind production data

  #We skips the first rows as they do not contain time series data
  wind_energy = wind_energy[2:]
  #Rename the 'kraftverknavn' column to 'timestamp' for clarity.
  wind_energy.rename(columns={'kraftverknavn': 'time'}, inplace=True)

  #Convert the 'timestamp' column to datetime format.
  wind_energy['time'] = pd.to_datetime(wind_energy['time'])

  #Create a new 'year' column by extracting the year from the 'timestamp' column.
  wind_energy['year'] = wind_energy.time.dt.year

  #Set the 'timestamp' column as the index for time-based analysis.
  wind_energy.set_index("time", inplace=True)

  # Display the first few rows of the structured data for inspection.
  wind_energy.head(100)

  #Choosing production data for choosen wind farms and saving into separate file
  wind_energy_chosen =wind_energy[chosen_windfarms]

  wind_energy_chosen.to_csv('nve_prod_chosen_2023.csv', index=True)


# Grouping hourly data into monthly data, by summing. Counting is also an option
def period_resampled_data(df,on_column,period = 'M', hours = None):
  if hours is None:
    df_monthly = df[str(on_column)].resample(period).agg(['sum'])
    df_monthly.columns = ["Production"]
  else:
    df_monthly = df[str(on_column)].resample(period).agg(['sum', 'count'])
    df_monthly.columns = ["Production", "Hours"]
  return df_monthly

#Finding first valid production hour and the hour one year later. This latter is used as first production hour.
def first_valid_hour(df,column = None):
  df = pd.DataFrame(df)
  if column is None:
    column = df.columns[0]

  #Finding first production hour
  index_of_first_valid_value = df[str(column)].first_valid_index()
  one_year = str(pd.to_datetime(index_of_first_valid_value) + pd.Timedelta(365, unit="d"))

  return index_of_first_valid_value, one_year

#Creating production index
def create_production_index(df_measured, df_simulated):
  production_index = df_measured.div(df_simulated)
  production_index.columns = ['Production Index']
  return production_index

In [ ]:
#Selecting n years of the production index dataframe and returning this

def select_period(df,num_years):
  _,one_year = first_valid_hour(df)

  end_date = pd.to_datetime(one_year) + pd.Timedelta(365 * num_years, unit="d")

  #greater than the start date and smaller than the end date
  selected_df = df[one_year:end_date]

  return selected_df

# Construction of weather data

In [ ]:
#Combining climate data set into one file
def combine_nc_dataset(name):
  full = xr.open_mfdataset(name + '_*_*.nc',combine = 'nested', concat_dim="time")
  full.to_netcdf(name + '_combined.nc')

#Modified version of Jesper Frausig´s code
#Construct columns from the downloaded parameters in the climate data.

def construct_columns(nc_name, ds):
  da = ds[nc_name]
  weights = np.cos(np.deg2rad(da.latitude))
  weights.name = "weights"
  t2m_C_weighted = da.weighted(weights)

  data_values=t2m_C_weighted.mean(["longitude", "latitude"])
  df = pd.DataFrame({"date": data_values.time, "value": data_values})
  df['timestamp']=df.date
  df.set_index('timestamp', inplace=True)
  df.drop("date", axis=1, inplace=True)
  nc_variable=df
  return nc_variable

#Modified version of Jesper Frausig´s code
#Calculating the direction of the wind speed
def create_wind_direction_df(v10, u10):
  angle = np.arctan2(v10.value,u10.value)
  df_wind_direction=pd.DataFrame()
  df_wind_direction['wdir']= angle * (180/np.pi) + 180
  return df_wind_direction

#Creating a wind rose plot from wind speeds and direction
def create_windrose(wind):
  ax = WindroseAxes.from_ax()
  ax.bar(wind.wdir.to_numpy(), wind.wind_speed.to_numpy(), normed=True, opening=0.8, edgecolor='white')
  ax.set_legend()

#Constructing  a dataframe containg wind_speed, temperature, pressure and roughness_length. The time are used as index.
def construct_weather_data(u10, v10,t2m, sp,fsr):
  weather_data=pd.DataFrame()
  weather_data['wind_speed']=((u10.value)**2+(v10.value)**2)**(0.5)
  weather_data['temperature'] = t2m
  weather_data['pressure'] = sp
  weather_data['roughness_length'] = fsr

  weather_data['timestamp']=weather_data.index
  return weather_data

#Creating a dataframe suitable for windpowerlib. Height are adjusted for wind speed and temperature
def construct_weather_windpowerlib(weather_data):

  df_weather_windpowerlib = pd.DataFrame(weather_data[['wind_speed','temperature','pressure','roughness_length']])
  header =[['wind_speed','temperature','pressure','roughness_length'],[100,2,0, 0]]
  df_weather_windpowerlib.columns = header
  return df_weather_windpowerlib

# Windpowerlib functions

In [ ]:
#Creating a turbine object. If the turbine type exist in windpowerlib, it uses that object.
#If not, it creats a new object using a defined function below.
def create_turbine_object(vindpark):
  df = wt.get_turbine_types(print_out=False)

  if df[df["turbine_type"].str.contains(vindpark['Model_2'])]['turbine_type'].any():
    my_turbine= WindTurbine(hub_height=vindpark["hub_height"],turbine_type=vindpark['Model_2'])

  else:
    my_turbine = create_new_turbine_object(vindpark)

  return my_turbine

In [ ]:
#Creating a new turbine objects using Windpowerlib if the turbine does not already exist in the library.
#Powercurve are created using a different defined punction
def create_new_turbine_object(Vindpark):
  new_turbine = {
      "turbine_type": Vindpark['Model_2'],
      "nominal_power": (Vindpark['Maksimal effekt'] *10**6),  # in W
      "hub_height": Vindpark['hub_height'],  # in m
      "power_curve": make_power_curve(external_power_curves['wind_speed'],  external_power_curves[Vindpark['Model_2']]),
  }
  new_turbine = WindTurbine(**new_turbine)

  return new_turbine

In [ ]:
#Creating power curves that does not exist in windpowerlib from external power curves
def make_power_curve(wind_speed, power):
  power_curve = create_power_curve(wind_speed = wind_speed, power = power)

  return power_curve

In [ ]:
#Simulating a tubine and using this tubine to simulated the rest of the farm
def windpowerlib_simulation(Vindpark,Vindpark_navn, weather_data):
    my_turbine = create_turbine_object(Vindpark)

    mc_my_turbine = ModelChain(my_turbine, wind_speed_model='logarithmic', hub_height=portfolio[Vindpark_navn]['hub_height'], density_correction=True).run_model(weather_data)
    my_turbine.power_output = mc_my_turbine.power_output
    simulated_data = wind_farm_simulated_power_output(my_turbine, Vindpark)


    return simulated_data, my_turbine

In [ ]:
#Modified version of Jesper Frausig´s code
def wind_farm_simulated_power_output(my_turbine, vindpark):

  # Extract power output data from the turbine model and store it in 'simulated_data'
  simulated_data=my_turbine.power_output

  # Localize the timezone of the simulated data and scale it
  # (by converting it to MW and adjusting for the number of operational turbines)
  simulated_data=simulated_data.tz_localize(None)
  simulated_data=simulated_data*(10**-6)*vindpark['Antall operative turbiner']

  # Convert simulated data to DataFrame for further manipulations
  simulated_data=pd.DataFrame(simulated_data)
  # Display the total sum of the simulated data for quick inspection
  print(simulated_data.sum())

  return simulated_data

In [ ]:
#Function for plotting power output
def plot_power_output(my_turbine, vindpark):
  my_turbine.power_output.plot(legend=True, label= vindpark['Model_2'])
  plt.xlabel('Time')
  plt.ylabel('Power in MW')
  plt.show()

In [ ]:
#Plotting power coefficient curve
#This code is a modefied version of the code from Windpowerlib: https://windpowerlib.readthedocs.io/en/stable/modelchain_example_notebook.html
def plot_power_coefficient_curve(my_turbine, vindpark):
  if my_turbine.power_coefficient_curve is not None:
          my_turbine.power_coefficient_curve.plot(
              x='wind_speed', y='value', style='*',
              title= vindpark['Model_2'] +' power coefficient curve')
          plt.xlabel('Wind speed in m/s')
          plt.ylabel('Power Coefficient, Cp')
          plt.show()


#Plotting power curve
def plot_power_curve(my_turbine, vindpark):
  if my_turbine.power_curve is not None:
          power_curve = my_turbine.power_curve.plot(x='wind_speed', y='value', style='*',
                                      title= vindpark['Model_2'] +' power curve')
          plt.xlabel('Wind speed in m/s')
          plt.ylabel('Power in MW')
          plt.show()

In [ ]:
#Making an aggregated Power curve for the whole wind farm

def windfarm_powercurve(my_turbine, Vindpark):
  #Creating  powercurves
  power_curve_wind_farm = my_turbine.power_curve
  power_curve_wind_farm['value'] = power_curve_wind_farm['value'] * Vindpark['Antall operative turbiner']*(10**-6)
  return power_curve_wind_farm

# Distances

Functions to calculate distances between the wind farm and coordinate given in the climate dataset are in this section.

In [ ]:
#Getting coordinate to weather data
def get_weather_coords(ds):
  latitude = ds['latitude'].values[0]
  longitude = ds['longitude'].values[0]
  weather_coordinate = (latitude,longitude)
  return weather_coordinate

In [ ]:
#calculating distance between weather coordinate and wind farm coordinate
def get_df_distance(ds,Vindpark_navn):
  weather_coord = get_weather_coords(ds)

  latitude_windfarm = portfolio[Vindpark_navn]['Latitude']
  longitude_windfarm = portfolio[Vindpark_navn]['Longitude']

  wind_farm_coord = (latitude_windfarm, longitude_windfarm)

  distance_km = geopy.distance.geodesic(weather_coord, wind_farm_coord).km

  df_distance = pd.DataFrame({'Wind_farm': Vindpark_navn,
                              'Weather_coordinate': str(weather_coord),
                              'Windfarm_coordinate': str(wind_farm_coord),
                              'Distance': distance_km}, index = [0])
  return df_distance


# Data cleaning & detection of partial downtime

The functions in this section are used to identify partial downtime.


In [ ]:
#Adjusting data points that are larger than the maximum rated power to maximum rated power.
#Data points below zero are adjusted to zero and missing points are set to the average of the successor and predecessor
def data_cleaning(data_set, Vindpark_navn, Vindpark):

  data_set.loc[data_set[Vindpark_navn] < 0, Vindpark_navn] = 0

  data_set.loc[data_set[Vindpark_navn] > Vindpark['Maksimal effekt'], Vindpark_navn] = Vindpark['Maksimal effekt']
  data_set.loc[data_set[Vindpark_navn].isnull(), Vindpark_navn] = (data_set[Vindpark_navn].fillna(method='ffill')+data_set[Vindpark_navn].fillna(method='bfill'))/2
  return data_set

In [ ]:
#Downtime is defined as all points that have consequetive 3 hours of zero production when inside production interval (5-25 m/s)

def downtime(df, Vindpark_navn):
  #Finding all entries that are NOT equal to zero.
  mask_d = df[Vindpark_navn].ne(0)
  #Finding cumulative sum of using the masked function
  cumsum_d = df.groupby(mask_d.cumsum())[Vindpark_navn].transform('size')
  #Datapoints that are within operational wind speed and that have three consequetive zeros .
  df_downtime_zero_production = df.loc[((df['wind_speed'] > 5) & (df['wind_speed'] < 25)) & (cumsum_d.gt(3).mask(mask_d, False))]
  #Returning this downtime.
  return df_downtime_zero_production

In [ ]:
#Creating a power curve 80% of the size a the original. Furthermore it is shifted to the right making only abnormal points lie beneath the curve.
def create_80_power_curve(power_curve_wind_farm):
  new_power_curve = power_curve_wind_farm.copy()
  new_power_curve['wind_speed'] = new_power_curve['wind_speed'] +5
  new_power_curve['value'] = new_power_curve['value'] *0.8
  return new_power_curve


In [ ]:
#This function identifies production points that lie beneath a power curve
def find_points_under_powercurve(scatterplot, curve, vindpark_navn):
  x_curve = curve['wind_speed']
  y_curve = curve['value']
  # Filter points that are below the curve

  df_under_pc = pd.DataFrame(columns=['wind_speed', 'value'])

  for index, row in scatterplot.iterrows():
      if row[vindpark_navn] < np.interp(row['wind_speed'], x_curve, y_curve):
        new_row = pd.DataFrame({'wind_speed':row['wind_speed'], 'value': row[vindpark_navn]}, index = [index])
        df_under_pc = pd.concat([df_under_pc, new_row])

  return df_under_pc

In [ ]:
def cumulative_effect_wind_farm(Vindpark):
  #Adjusting powercurve to the entire wind farm
  num_turbines = Vindpark['Antall operative turbiner']
  df_cumulative_effect = pd.DataFrame({ 'Num_windturbines' : range(1, int(num_turbines) +1 ,1)})
  effect_per_turbine = Vindpark['Maksimal effekt']/Vindpark['Antall operative turbiner']
  df_cumulative_effect['Cumulative_effect'] = df_cumulative_effect.Num_windturbines * effect_per_turbine
  return df_cumulative_effect

In [ ]:
#Finding points that are located near a effect line
def find_neigbour_points(df_cumulative_effect,df_scatter):
  horizontal_line_y = df_cumulative_effect['Cumulative_effect']
  distance_threshold = 0.05

  df_neigbours = pd.DataFrame(columns = ['wind_speed', 'value', 'effect'])
  for line in horizontal_line_y:
    for index, row in df_scatter.iterrows():
        if abs(row['value'] - line) < distance_threshold:
          new_row = pd.DataFrame({'wind_speed':row['wind_speed'], 'value': row['value'],'effect': line}, index = [index])
          df_neigbours = pd.concat([df_neigbours, new_row])

  return df_neigbours


In [ ]:
#Removing partial downtime from the production data set
def remove_downtime_points(df_production, df_downtime):
  df_production = df_production.drop(df_downtime.index)
  return df_production

# Metrics, residuals and statistic tests

Metrics and residuals are calculated to the different models to chose the quality of the models as well as chosing the best model.

In [ ]:
#Creating three metrics to a wind farm using y_test and the predicted y
def get_metrics(vindpark_navn, y_test,y_pred):
  r2 = r2_score(y_test, y_pred)
  mse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  df_metrics = pd.DataFrame({'Wind Farm': vindpark_navn, 'R2': r2, 'MSE': mse, 'MAE': mae}, index = [vindpark_navn])

  return df_metrics

In [ ]:

#Creating different statistical tests to test the linear assumptions
def regression_assumptions(vindpark_navn,X_train, X_test,y_train, y_test, y_pred, df):
  sns.pairplot(df, x_vars='Time', y_vars='Production Index', height=7, aspect=0.7)
  residuals = pd.DataFrame(y_test - y_pred)

  lr_model = OLS(y_train, X_train).fit()
  rainbow_test = linear_rainbow(lr_model, frac = 0.5)
  ljungbox = smd.acorr_ljungbox(residuals,return_df=True)
  min_ljung_pvalue = min(ljungbox.lb_pvalue)

  df_tests = pd.DataFrame({'Ljungbox test': min_ljung_pvalue,'Rainbow test': rainbow_test[1],
                           'Shapiro-Wilk test':shapiro(df)[1],'Goldfeld Quandt':smd.het_goldfeldquandt(residuals, X_test)[1]}, index=[vindpark_navn])
  return df_tests

In [ ]:
#Calculating mean squared error and mean absolute error to the auto arima model
def forecast_metrics_arima(Vindpark_navn, forecast, actual):
  mse = np.mean((forecast - actual)**2) # MSE
  mae = np.mean(np.abs(forecast - actual))    # MAE

  arima_metrics = pd.DataFrame({'MSE': mse, 'MAE': mae}, index = [Vindpark_navn])
  return arima_metrics


# Regression models

The modeling phase consists of creating the different algorithms used in the modeling as well as

In [ ]:
#Function to sleect type of regression, normal, seasonal or limited normal
def methods_regression(simulated_data,valid_energy_prod,Vindpark_navn, type_reg, num_years = 0):
  #Translite if-else

  if type_reg == "Month":
      period = 'M'
  elif type_reg =="Season":
    period = 'W'
  else:
    period = 'M'

  #Grouping simulated and measured production data into a specific period, mostly months.
  df_period_simulated = period_resampled_data(simulated_data,'feedin_power_plant',period, hours = None)
  df_period_measured = period_resampled_data(valid_energy_prod,Vindpark_navn, period, hours = None)

  #Creating a production indec
  df_production_index_periodly = create_production_index(df_period_measured, df_period_simulated)

  #If true, only first n years of data are used
  if num_years != 0:
    df_production_index_periodly = select_period(df_production_index_periodly, num_years)

  #Models for monthly data
  if type_reg =='Month':
    #Running the three models
    arima_slope, arima_metrics = auto_arima(df_production_index_periodly, Vindpark_navn)
    results, lr_metrics, ran_metrics, lr_residuals,ran_residuals = reg_RANSAC(df_production_index_periodly,Vindpark_navn,'Plot')

    #Getting slipes
    lin_slope = results['Linear_coefficient']
    ransac_slope = results['Ransac_coefficient']

    #Estimating yearly degradation tto the models
    yearly_degradation_lin = 12 * lin_slope
    yearly_degradation_ransac = 12 * ransac_slope
    yearly_degradation_arima = 12 * arima_slope

    #this is sed to create a results df that are returned
    results_degrad = pd.DataFrame({'Wind_farm': str(Vindpark_navn),
                'Linear_degradation': round(float(yearly_degradation_lin),4),
                'Ransac_degradation': round(float(yearly_degradation_ransac),4),
                'Arima degradation': round(float(yearly_degradation_arima),4)}, index=[0])

    return results_degrad, lr_metrics, ran_metrics,arima_metrics, lr_residuals,ran_residuals

  #Models for seasonal regression
  elif type_reg == 'Season':
    #Finding the specific years of production
    unique_years = pd.DataFrame(df_production_index_periodly.index.year).drop_duplicates()['time'].tolist()
    df_production_index_periodly['quarters'] = df_production_index_periodly.index.quarter

    #Making dataframes containg all four quarters and all years from unique years.
    quarters = {'Q1': 1,'Q2': 2,'Q3': 3,'Q4': 4}
    df_year_lr = pd.DataFrame(columns = ['Year','Q1','Q2','Q3','Q4' ])
    df_year_ran = pd.DataFrame(columns = ['Year','Q1','Q2','Q3','Q4' ])
    for year in unique_years:
      year_row_lr = pd.DataFrame({'Year': year,
                  'Q1': None,
                  'Q2': None,
                  'Q3': None,
                  'Q4': None}, index = [0])

      year_row_ran = pd.DataFrame({'Year': year,
                  'Q1': None,
                  'Q2': None,
                  'Q3': None,
                  'Q4': None}, index = [0])

      #Iterating over each quarter and running regression models
      df_new = df_production_index_periodly.iloc[(df_production_index_periodly.index.year == year)]
      for quarter in quarters:
        df_q = df_new.iloc[df_new['quarters'].values == quarters[quarter]]

        if df_q.shape[0] > 10:
          results_degrad, lr_metrics, ran_metrics = reg_RANSAC(df_q)
          year_row_lr.loc[year_row_lr[quarter].values == None, quarter] = results_degrad['Linear_coefficient']
          year_row_ran.loc[year_row_ran[quarter].values == None, quarter] = results_degrad['Ransac_coefficient']

      #These are added to the data frame
      df_year_lr = pd.concat([df_year_lr,year_row_lr], ignore_index=True)
      df_year_ran = pd.concat([df_year_ran, year_row_ran], ignore_index=True)

    #The average degradation to each quarter are calculated and return in dataframe format.
    df_avg_wind_lr = pd.DataFrame({'Q1': df_year_lr['Q1'].mean(),
                                  'Q2': df_year_lr['Q2'].mean(),
                                  'Q3': df_year_lr['Q3'].mean(),
                                  'Q4': df_year_lr['Q4'].mean()}, index = [Vindpark_navn])
    df_avg_wind_ran = pd.DataFrame({'Q1': df_year_ran['Q1'].mean(),
                                'Q2': df_year_ran['Q2'].mean(),
                                'Q3': df_year_ran['Q3'].mean(),
                                'Q4': df_year_ran['Q4'].mean()}, index = [Vindpark_navn])
    return df_avg_wind_lr,df_avg_wind_ran, lr_metrics, ran_metrics


In [ ]:
#Regression function that conducts normal linear regression and Ransac regression. Returining, metrics, tests, coefficients and plotting the results
def reg_RANSAC(df, vindpark_navn = None, regression_plot = None):
  #Copying tthe production index data and adding a dummy variable for regression
  df = df.copy()
  df['Time'] = np.arange(len(df.index))


  # Dividing into  target and feature data
  X = df.loc[:, ['Time']].values  # features
  y = df.loc[:, ['Production Index']].values  # target

  #Splitting into training and test set
  X_train, X_test, y_train, y_test = train_test_split(X, y,random_state = 0,test_size=0.20, shuffle=False)

  #Making parameter grid for grid search
  if X_train.shape[0] > 25:
    param_grid = {'min_samples': [20,25,30,40],
                'residual_threshold':[0.375, 0.5,0.625, 0.75,1.0,1.5,2.0,2.5,3.0]}
  else:
    param_grid = {'min_samples': [4,6,8,10,12],
                'residual_threshold':[0.375, 0.5,0.625, 0.75,1.0,1.5,2.0,2.5,3.0]}

  #Fitting and predicting a Linear regression model
  lr = LinearRegression()
  lr.fit(X_train, y_train)
  y_pred_lr = lr.predict(X_test)

  #Creating a ransac regression model
  ransac = RANSACRegressor(LinearRegression(),
                          max_trials = 100,
                          loss = 'absolute_error',
                          random_state = 0)

  #Fitting a grid search
  gs = GridSearchCV(estimator=ransac, param_grid = param_grid, n_jobs=-1)
  gs.fit(X_train, y_train)


  print(" Results from Grid Search " )
  print("\n The best estimator across ALL searched params:\n",gs.best_estimator_)
  print("\n The best score across ALL searched params:\n",gs.best_score_)
  print("\n The best parameters across ALL searched params:\n",gs.best_params_)

  #Best parameters from gridsearch are used to fit a new ransac model. Prediction are also made
  ransac = gs.best_estimator_

  ransac.fit(X_train, y_train)
  y_pred_ran = ransac.predict(X_test)


  #Plotting the results, marking inliers, outliers and regression lines for both models.
  if vindpark_navn is not None:
    lr_residuals = regression_assumptions(vindpark_navn,X_train,X_test,y_train, y_test, y_pred_lr, df)
    ran_residuals = regression_assumptions(vindpark_navn,X_train,X_test, y_train,y_test, y_pred_ran, df)

  lr_metrics = get_metrics(vindpark_navn, y_test,y_pred_lr)
  ran_metrics = get_metrics(vindpark_navn, y_test,y_pred_ran)

  inlier_mask = ransac.inlier_mask_
  outlier_mask = np.logical_not(inlier_mask)


  #Predict data of estimated models
  line_X_test = np.arange(X_test.min(), X_test.max())[:, np.newaxis]
  line_y_test = lr.predict(line_X_test)
  line_y_ransac_test = ransac.predict(line_X_test)

  line_X_train = np.arange(X_train.min(), X_train.max())[:, np.newaxis]
  line_y_train = lr.predict(line_X_train)
  line_y_ransac_train = ransac.predict(line_X_train)


  # Compare estimated coefficients
  lr_coef = lr.coef_
  ransac_coef = ransac.estimator_.coef_

  #plotting results from linear and ransac model. This code is based on the code from: https://scikit-learn.org/stable/auto_examples/linear_model/plot_ransac.html
  if regression_plot is not None:
    lw = 2
    plt.scatter(
        X_train[inlier_mask], y_train[inlier_mask], color="yellowgreen", marker=".", label="Inliers"
    )
    plt.scatter(
        X_train[outlier_mask], y_train[outlier_mask], color="gold", marker=".", label="Outliers"
    )

    plt.plot(line_X_train, line_y_train, color="navy", linewidth=lw, label="Linear regressor")
    plt.plot(
        line_X_train,
        line_y_ransac_train,
        color="cornflowerblue",
        linewidth=lw,
        label="RANSAC regressor",
    )

    plt.legend(loc="lower left")
    plt.xlabel("Time")
    plt.ylabel("Performance Index")
    plt.show()

  #Creating a results df for returning coefficients
  results  = pd.DataFrame({'Linear_coefficient': round(float(lr_coef),4),
                            'Ransac_coefficient': round(float(ransac_coef),4)}, index=[vindpark_navn])
  #Returning coefficients
  if vindpark_navn is None:
    return results, lr_metrics, ran_metrics
  else:
    return results, lr_metrics, ran_metrics, lr_residuals,ran_residuals


In [ ]:
def auto_arima(df, vindpark_navn = None, regression_plot = None):
  #copying the production index df
  df_Arima = df.copy()
  #adding a dummy column from to use for regression
  df_Arima['Time'] = np.arange(len(df_Arima.index))

  df_Arima['DateTimeIndex']= pd.to_datetime(df_Arima.index)
  df_Arima = df_Arima.set_index(keys=['DateTimeIndex'])

  # Training data
  dataset_len = df_Arima.shape[0]
  split_index = round(dataset_len*0.9)
  train_set_end_date = df_Arima.index[split_index]

  #dividing into training and test set
  df_train = df_Arima.loc[df_Arima.index <= train_set_end_date].copy()
  df_test = df_Arima.loc[df_Arima.index > train_set_end_date].copy()

  #dividing into feature and target value
  y_train = pd.DataFrame(df_train['Production Index'])
  X_train = pd.DataFrame(df_train['Time'])  # Extract the index column into a DataFrame

  y_test = pd.DataFrame(df_test['Production Index'])
  X_test = pd.DataFrame(df_test['Time'])

  #Modeling using ordinary least squares and plotting different diagnostic plots
  model_lr = OLS(y_train, X_train)
  results_lr = model_lr.fit()
  rainbow_test = linear_rainbow(results_lr)
  residuals = pd.DataFrame(results_lr.resid)

  residuals_diff_1 = residuals.diff().dropna()
  residuals_diff_2 = residuals_diff_1.diff().dropna()
   # autocorrelation
  sm.graphics.tsa.plot_acf(residuals,alpha=0.05)
  plt.show()

  sm.graphics.tsa.plot_acf(residuals.diff().dropna(), alpha=0.05)
  plt.show()

  sm.graphics.tsa.plot_acf(residuals_diff_2, alpha=0.05)
  plt.show()

  #Plotting decomposed data
  components = seasonal_decompose(residuals)
  components.plot()


  olsr_resid_diff_1_24 = residuals_diff_1.diff(periods=24)

   # partial autocorrelation
  sm.graphics.tsa.plot_pacf(residuals)
  plt.show()


  # Running sarimx auto arima model
  sxmodel = pm.auto_arima(y_train, X_train,
                            start_p=1, start_q=1,
                            test='adf',
                            max_p=3, max_q=3, m=12,
                            start_P=0, seasonal=True,
                            d=None, D=1, trace=True,
                            error_action='ignore',
                            suppress_warnings=True,
                            stepwise=True)

  #Plotting diagnostic
  sxmodel.plot_diagnostics(figsize=(10,8))
  plt.show()
  #Plotting results
  print(sxmodel.summary())

  #Getting time coefficientt
  model_dict = sxmodel.to_dict()
  coefficient = model_dict['params']['Time']

  #Predicting forcast
  prediction = sxmodel.predict(n_periods=len(y_test),X = X_test)
  prediction = pd.DataFrame(prediction,index = y_test.index,columns=['Prediction'])

  #Calculating. arima errors.
  arima_metrics = forecast_metrics_arima(vindpark_navn, prediction['Prediction'], y_test['Production Index'])

  #plot the predictions for validation set
  plt.plot(y_train, label='Train')
  plt.plot(y_test, label='Actual')
  plt.plot(prediction, label='Prediction')
  plt.legend(loc="lower right")
  plt.show()

  return coefficient, arima_metrics



# Plotting functions

Functions in this section are for plotting different data in the analysis.

This function plots the production against wind resources. Additionally, the power curve, a shifted 80% power curve and pre-efined effectlines are plotted. Data points beneath the 80% PC are marked in a green color. Furthermore, those points that are also close to the effectlines are marked in a yellow color. These are considered part of the partial downtime.

In [ ]:
#Bar chart showing the count of different turbine types selected in this thesis
def plot_distribution_turbines(portfolio_data):
  chart = alt.Chart(portfolio_data).mark_bar().encode(
      x=alt.X('Model_2:O', title='Model name'),
      y=alt.Y('Antall:Q', title='Number of turbines'),
      color = 'Manufacture_year:O',
      order=alt.Order("Manufacture_year:O", sort='ascending'),
  ).configure_axis(
      labelFontSize=15,
      titleFontSize=20).configure_axis(
      labelFontSize=13,
      titleFontSize=14)
  display(chart)




In [ ]:
#Plotting bar chart of wind turbine dimensions using Altair
def plot_wind_turbine_dimensions(portfolio_data):
  chart_hub_height = alt.Chart(portfolio_data).mark_bar().encode(
      x=alt.X('Model_2:O',title='Model name'),
      y=alt.Y('hub_height:Q',title='Hub height [m]'),
      color = 'Manufacture_year:O'
  ).properties(
      title='Hub height'
  )

  chart_rotor_diameter = alt.Chart(portfolio_data).mark_bar().encode(
      x=alt.X('Model_2:O',title='Model name'),
      y=alt.Y('rotordiameter:Q',title='Rotor diameter [m]'),
      color = 'Manufacture_year:O').properties(
      title='Rotor diameter'
  )


  chart_generator_performance = alt.Chart(portfolio_data).mark_bar().encode(
      x=alt.X('Model_2:O',title='Model name'),
      y=alt.Y('Generator_performance:Q',title='Generator performance [MW]'),
      color = 'Manufacture_year:O').properties(
      title='Generator performance'
  )
  chart = alt.hconcat(chart_hub_height, chart_rotor_diameter, chart_generator_performance).properties(
  ).configure_axis(
      labelFontSize=13,
      titleFontSize=14)

  display(chart)

In [ ]:
#Plottingthe power curve, a shifted 80% power curve and pre-efined effectlines are plotted. Data points beneath the 80% PC are marked in a green color.
#Furthermore, those points that are also close to the effectlines are marked in a yellow color.

def plot_production_powercurve(Vindpark_navn,df_weather_downtime,df_under_pc,df_neigbours,df_cumulative_effect,power_curve_wind_farm,new_power_curve):
  plt.figure()
  plt.scatter(df_weather_downtime['wind_speed'], df_weather_downtime[Vindpark_navn], marker='.', alpha=0.7, label = 'Power production')
  plt.scatter(df_under_pc['wind_speed'], df_under_pc['value'], color='yellowgreen', label='Filtered Points', marker='.', alpha=0.7)
  plt.scatter(df_neigbours['wind_speed'], df_neigbours['value'], color='gold', label='Points Close to Horizontal Line')
  plt.hlines(df_cumulative_effect['Cumulative_effect'], xmin = 0, xmax = 25, colors= 'r', label = 'Cumulative effect lines')
  plt.plot(power_curve_wind_farm['wind_speed'],power_curve_wind_farm['value'],color= 'navy', label = 'Power curve')
  plt.plot(new_power_curve['wind_speed'],new_power_curve['value'],color= 'cornflowerblue', label = 'Power curve (80%)')
  plt.title('Production plot')
  plt.xlabel('Wind speed [m/s]')
  plt.ylabel('Power production [MW]')
  plt.legend(loc = 'upper right', bbox_to_anchor=(1.55, 1.0))
  plt.show()

The function below are used to plot a power curve and a power coefficient curve of one turbine in the same plot.

In [ ]:
#Plotting power curve and power coefficientt curve to a turbien in the same plot
def plot_power_and_coeffcient_curve(my_turbine):
  if my_turbine.power_coefficient_curve is not None and my_turbine.power_curve is not None:
    fig, ax1 = plt.subplots()

    ax1.set_xlabel('Wind speed in m/s')
    ax1.set_ylabel('Cp', color='r')
    ax1.plot(my_turbine.power_coefficient_curve['wind_speed'], my_turbine.power_coefficient_curve['value'], color= 'r', label = 'Power Coefficient Curve')
    ax1.tick_params(axis='y', labelcolor='r')
    ax1.legend(loc= 'lower right')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    ax2.set_ylabel(('Power in MW'), color='b')
    ax2.plot(my_turbine.power_curve['wind_speed'], my_turbine.power_curve['value'], color='b', label='Power curve')
    ax2.tick_params(axis='y', labelcolor='b')
    ax2.legend(loc=0)
    fig.tight_layout()
    plt.show()


In [ ]:
#make_weighted_degradation is used to calculate average degradation and
#weighted average estimation to the different regression models used in this thesis.

def make_weighted_degradation(df_result, seasons = False):
  proportion_effect = pd.DataFrame(index = portfolio_data.index)
  proportion_effect['Maksimal effekt'] = portfolio_data['Maksimal effekt']
  tot_effect = portfolio_data['Maksimal effekt'].sum()

  proportion_effect['Proportion'] = proportion_effect['Maksimal effekt']/tot_effect

  selected_wind_farms = proportion_effect.loc[proportion_effect.index.isin(df_result.index)]

  if seasons == False:
    df_weight = pd.DataFrame()
    df_weight['Weighted_Linear_degradation'] = df_result['Linear_degradation'].values * selected_wind_farms['Proportion']
    df_weight['Weighted_Ransac_degradation'] = df_result['Ransac_degradation'].values * selected_wind_farms['Proportion']
    df_weight['Weighted_Arima_degradation'] = df_result['Arima degradation'].values * selected_wind_farms['Proportion']



    lr_weighted_average = df_weight['Weighted_Linear_degradation'].sum()
    ran_weighted_average = df_weight['Weighted_Ransac_degradation'].sum()
    arima_weighted_average = df_weight['Weighted_Arima_degradation'].sum()

    df_results_avg = pd.DataFrame({'Linear Reg Mean': df_result['Linear_degradation'].mean(),
                                  'Ransac Reg Mean':df_result['Ransac_degradation'].mean(),
                                  'Arima Reg Mean':df_result['Arima degradation'].mean(),
                                  'Linear Reg Weighted Avg':lr_weighted_average,
                                  'Ransac Reg Weighted Avg': ran_weighted_average,
                                  'Arima Reg Weighted Avg': arima_weighted_average}, index = [0])


  elif seasons == True:
    df_weight = pd.DataFrame()
    df_weight['Weighted_Q1_degradation'] = df_result['Q1'].values * selected_wind_farms['Proportion']
    df_weight['Weighted_Q2_degradation'] = df_result['Q2'].values * selected_wind_farms['Proportion']
    df_weight['Weighted_Q3_degradation'] = df_result['Q3'].values * selected_wind_farms['Proportion']
    df_weight['Weighted_Q4_degradation'] = df_result['Q4'].values * selected_wind_farms['Proportion']

    q1_weighted_average = df_weight['Weighted_Q1_degradation'].sum()
    q2_weighted_average = df_weight['Weighted_Q2_degradation'].sum()
    q3_weighted_average = df_weight['Weighted_Q3_degradation'].sum()
    q4_weighted_average = df_weight['Weighted_Q4_degradation'].sum()

    df_results_avg = pd.DataFrame({'Q1' :results_season_ran['Q1'].mean() ,
                          'Q2' :results_season_ran['Q2'].mean() ,
                          'Q3' :results_season_ran['Q3'].mean() ,
                          'Q4' :results_season_ran['Q4'].mean() },index = ['Average Degradation'])

    df_results_weighted_avg = pd.DataFrame({'Q1' :q1_weighted_average ,
                          'Q2' :q2_weighted_average ,
                          'Q3' :q3_weighted_average ,
                          'Q4' :q4_weighted_average },index = ['Weighted Average Degradation'])

    df_results_avg = pd.concat([df_results_avg,df_results_weighted_avg])


  return df_results_avg, df_weight


In [ ]:
#Making a combined confidence interval to the final result
def mean_confidence_interval(data_1,data_2, confidence=0.95):
  n1 = results_without_downtime['Ransac_degradation'].shape[0]
  n2 = df_weighted['Weighted_Ransac_degradation'].shape[0]

  df = n1 + n2-2

  mean_1 = np.mean(data_1)
  mean_2 = np.sum(data_2)
  total_mean = (0.5 * mean_1 + 0.5 * mean_2)

  std_err_1 = scipy.stats.sem(data_1)
  std_err_2 = scipy.stats.sem(data_2)

  total_std_error = 0.5 * math.sqrt(((std_err_1**2)/n1 + (std_err_2**2))/n2)


  pooled_standard_deviation = math.sqrt(
                      ((n1 - 1)*std_err_1 * std_err_1 +
                     (n2-1)*std_err_2 * std_err_2) /
                                  (n1 + n2-2))

  interval = total_std_error * scipy.stats.t.ppf((1 + confidence) / 2., df)
  return interval





# Functions for testing for differences

The functions beneath are used to test if there is a significant difference between different groups defined in the thesis. A Mannwhitney u-test are used as the groups are of small sample sizes.

In [ ]:
#Running one manwhitney u-test. This returns the groups tested together with test-statistic and p-value
def mannwhitney_test(groups, reg_type):
  group = list(groups.items())
  group_1_name = group[0][0]
  group_2_name = group[1][0]
  group_1_values = group[0][1]
  group_2_values = group[1][1]
  U1, p = mannwhitneyu(group_1_values,group_2_values , method='auto')

  df_mann_whitneyu = pd.DataFrame({'Test_group_1':group_1_name,'Test_group_2':group_2_name,'Test-statistic':U1, 'p-value' : p}, index = [reg_type])

  return df_mann_whitneyu



In [ ]:
#This functions allow the program to iterate over multiple tests in the same testing category.
def run_u_test_case(cases):
  reg_type = cases[0]
  test_groups = cases[1]
  df_test = pd.DataFrame()
  for groups in test_groups:

    df = mannwhitney_test(groups, reg_type)
    df_test = pd.concat([df_test,df])
  return df_test



## Chosen windfarms

Wind farms are chosen in the section below. Wind farm data are cleaned to ensure the correctness of the data.

In [ ]:
#Choosing wind farms
chosen_windfarm = {'Egersund':'Egersund','Fakken':'Fakken', 'Hamnefjell':'Hamnefjell','Hitra':'Hitra',  'Kjøllefjord':'Kjollefjord', 'Lista':'Lista', 'Raggovidda':'Raggovidda', 'Rye Vind':'ryevind', 'Røyrmyra':'royrmyra','Sandøy':'Sandoy','Skomakerfjellet':'Skomakerfjellet','Tellenes':'Tellenes', 'Valsneset':'Valsneset','Ytre Vikna':'Ytre_Vikna', 'Åsen II':'Asen2'}

In [ ]:
# Read the wind turbine data from a CSV file
portfolio_data = pd.read_csv("https://pvexpect.com/Vind/Vindturbine_portfolio_2.csv", delimiter=";")

In [ ]:
#Importing external power_curves and power coefficient curves that was not in windpowerlib
external_power_curves = pd.read_csv('PowerCurves.csv',delimiter=";")

external_power_coeffcient_curves = pd.read_csv('Power_coefficient_curves.csv',delimiter=",")

In [ ]:
#Selecting data reagarding the chosen wind farms
portfolio_data = portfolio_data.loc[portfolio_data['Navn'].isin(chosen_windfarm)]
portfolio_data.head(16)

In [ ]:
#Adding a new column with the geografic cluster are defined based on price zones.
portfolio_data['Geografic_area'] = None
portfolio_data.loc[portfolio_data['Elspotområde'] == 'NO 2','Geografic_area'] = 'South_west'
portfolio_data.loc[portfolio_data['Elspotområde'] == 'NO 3','Geografic_area'] = 'Mid'
portfolio_data.loc[portfolio_data['Elspotområde'] == 'NO 4','Geografic_area'] = 'North'

#Added two columns; one with manufacture year and one with rated power of one turbine
portfolio_data['Manufacture_year'] = [2014,2002,2004,2004,2004,2005,2011,1989,2004,1997,2015,2014,2005,2005,2004]
portfolio_data['Generator_performance'] = [3.4,3,3.45,2.3,2.3,2.3,3,0.225,0.8,0.750,3.3,3.2,2.3,2.3,0.8]

#Adding a column classifying newer and older wind turbines
portfolio_data['Age_groups'] = None
portfolio_data.loc[portfolio_data['Manufacture_year'] >= 2011,'Age_groups'] = 'New_turbine'
portfolio_data.loc[portfolio_data['Manufacture_year'] < 2011,'Age_groups'] = 'Old_turbine'

#Extract numerical values from 'Gjennomsnitt navhøyde' column and fill missing values with -1
#Modifisert versjon av Jesper Frausig sin kode
portfolio_data['hub_height'] = portfolio_data['hub_height'].str.extract('(\d+)').fillna(-1).astype(int)
portfolio_data['rotordiameter'] = portfolio_data['rotordiameter'].str.extract('(\d+)').fillna(-1).astype(int)
#Renaming coordinate colum
portfolio_data.rename(columns={"Lengdegrad": "Latitude", "Breddegrad": "Longitude"},inplace=True)


#Correcting wind farm data that were wrong
portfolio_data.loc[portfolio_data['Navn'] == 'Åsen II','Model_2'] = 'E48/800'
portfolio_data.loc[portfolio_data['Navn'] == 'Rye Vind','Model_2'] = 'V27/225'

portfolio_data.loc[portfolio_data['Navn'] == 'Hitra','Model'] = 'SWT-2.3-82'
portfolio_data.loc[portfolio_data['Navn'] == 'Hitra','Model_2'] = 'SWT82/2300'


portfolio_data.loc[portfolio_data['Navn'] == 'Sandøy','Model_2'] = 'NM48/750'
portfolio_data.loc[portfolio_data['Navn'] == 'Sandøy','hub_height'] = 50
portfolio_data.loc[portfolio_data['Navn'] == 'Hitra','hub_height'] = 70

portfolio_data.loc[portfolio_data['Navn'] == 'Sandøy','rotordiameter'] = 48.2
portfolio_data.loc[portfolio_data['Navn'] == 'Hitra','rotordiameter'] = 82.4



#Removed columns that were unnecessary
portfolio_data.drop(columns =['Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28','Unnamed: 30','Høyde', 'Unnamed: 31','Unnamed: 33','Antall.2',	'Idriftsatt.2','Turbinprodusent.2',	'Turbintype.2',
                              'Antall.1',	'Idriftsatt.1','Turbinprodusent.1','Turbintype.1'], inplace = True)

#Creating dictionare of the data
portfolio = {
    turbin: row.to_dict()
    for turbin, row in portfolio_data.set_index("Navn").iterrows()
}
#Setting 'Navn' as index column
portfolio_data.set_index('Navn', inplace = True)
portfolio_data.index.names = ['Wind_farm']

In [ ]:
#plottting the count of different wind turbines in used in this thesis
plot_distribution_turbines(portfolio_data)

In [ ]:
#Plotting the dimensions of the turbines
plot_wind_turbine_dimensions(portfolio_data)

In [ ]:
#Modified version of Jesper Frausig´s code

# Define a variable 'norway_counties' and load geojson data for European countries
norway_counties = alt.topo_feature('https://dmws.hkvservices.nl/dataportal/data.asmx/read?database=vega&key=europe', 'europe')

# Create a base map of Norway using the geojson data.
norway = alt.Chart(norway_counties).mark_geoshape(stroke="black", fill="lightgrey").encode(
).project(
    type='mercator',
    scale=1000,
    center=[16, 65],
).properties(
    width=600,
    height=800
)

# Create a map of wind farms using 'portfolio_data' with circles representing wind turbines. Geografic clusters are marked in different colors.
south_west = alt.Chart(portfolio_data.loc[portfolio_data['Geografic_area'] =='South_west']).mark_circle(size=30, stroke="blue", fill="blue").encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
).project(
    type='mercator',
    scale=1000,
    center=[16, 65],
).properties(
    width=600,
    height=800
)

mid = alt.Chart(portfolio_data.loc[portfolio_data['Geografic_area'] =='Mid']).mark_circle(size=30, stroke="red", fill="red").encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
).project(
    type='mercator',
    scale=1000,
    center=[16, 65],
).properties(
    width=600,
    height=800
)

north = alt.Chart(portfolio_data.loc[portfolio_data['Geografic_area'] =='North']).mark_circle(size=30, stroke="green", fill="green").encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
).project(
    type='mercator',
    scale=1000,
    center=[16, 65],
).properties(
    width=600,
    height=800
)


# Combine the maps by adding them together
norway + south_west +mid + north


In [ ]:
#Running one wind farm
def run_one_windfarm(selected_windfarm, power_curve = None, regression_curve = None):

  #Choose the windfarm we want to analyze
  Vindpark_navn = selected_windfarm
  file_wind_farm = chosen_windfarm[Vindpark_navn]
  Vindpark= portfolio[Vindpark_navn]
  Vindpark_start_dato = Vindpark["Idriftsatt"]


  #Combine all the data into one file
  combine_nc_dataset(file_wind_farm)

  #Opening weather file
  ds = xr.open_dataset(file_wind_farm +'_combined.nc')

  #Import production data
  if Vindpark_navn == 'Sandøy':
    production_data = pd.read_csv('nve_prod_chosen.csv')
    production_data.set_index('time',inplace=True)
  else:
    production_data = pd.read_csv('nve_prod_chosen_2023.csv')
    production_data.set_index('time',inplace=True)


  #Creating columns from imported weather data
  u10 = construct_columns('u100',ds)
  v10 = construct_columns('v100',ds)
  fg10 = construct_columns('i10fg',ds)
  t2m = construct_columns('t2m', ds)
  fsr = construct_columns('fsr',ds)
  sp = construct_columns('sp',ds)

  columns = [u10,v10,fg10,t2m, fsr, sp]

  #Creating distance dataframe
  df_distance = get_df_distance(ds, Vindpark_navn)

  #Creating direction
  direction = create_wind_direction_df(v10, u10)

  #Finding first production hour and the same hour one year later. The latter is the selected startpoint
  _ , one_year = first_valid_hour(production_data[Vindpark_navn])
  direction = direction[one_year:]
  #Create weather dataframe

  #Constructing weatherdata dataframe
  weather_data = construct_weather_data(u10, v10,t2m, sp,fsr)

  #Selecting productiondata for one wind farm
  energy_prod = production_data[[Vindpark_navn]]

  #Creating a dataframe with valid productionhours
  start_hour, one_year = first_valid_hour(energy_prod)

  #Selecting relevant parts of the data
  valid_energy_prod = energy_prod[one_year:]
  valid_energy_prod.index = pd.to_datetime(valid_energy_prod.index)

  fg10.rename({'value': 'fg10'}, axis=1, inplace=True)

  #Creating a common dataframe with weather and producction data among otther data
  valid_weather_data = weather_data[one_year:]
  wind=pd.DataFrame()
  wind=pd.concat([valid_weather_data,valid_energy_prod, direction, fg10], axis=1)
  wind.dropna(inplace=True)

  #Creating a windrose showing wind direction
  create_windrose(wind)

  #Calculating cumulative maximum effect
  df_cumulative_effect =  cumulative_effect_wind_farm(Vindpark)

  #Finding rows considered downtime
  df_weather_downtime = wind[['wind_speed', Vindpark_navn]]
  df_downtime_zero_production = downtime(df_weather_downtime, Vindpark_navn)

  #Making a dataframe suitable to windpowerlib
  df_weather_windpowerlib = construct_weather_windpowerlib(valid_weather_data)

  #remove zero entries
  df_weather_windpowerlib_without_zero_downtime = remove_downtime_points(df_weather_windpowerlib, df_downtime_zero_production)
  df_valid_prod_without_zero_downtime = remove_downtime_points(valid_energy_prod, df_downtime_zero_production)

  #Simulating data
  simulated_data, my_turbine = windpowerlib_simulation(Vindpark,Vindpark_navn, df_weather_windpowerlib)
  plot_power_and_coeffcient_curve(my_turbine)

  #Creating a dataframe with differnt data statistics and checking for irregularties in the data
  df_data_statistics = pd.DataFrame({'Observasjoner tot prod': production_data[Vindpark_navn].shape[0],
                                    'Observasjoner valid prod':valid_energy_prod.shape[0] ,
                                    'Missing values prod':valid_energy_prod.isnull().sum(),
                                    'Negative values prod':(valid_energy_prod < 0).values.sum(),
                                    'Zero values prod':(valid_energy_prod == 0).values.sum(),
                                    'Prod over capacity':(valid_energy_prod >Vindpark['Maksimal effekt']).sum(),
                                    'Observasjoner df_windpowerlib':df_weather_windpowerlib.shape[0] ,
                                    'Missing values wind_peed':df_weather_windpowerlib['wind_speed'].isnull().sum(),
                                    'Missing values temperature':df_weather_windpowerlib['temperature'].isnull().sum(),
                                     'Missing values pressure':df_weather_windpowerlib['pressure'].isnull().sum(),
                                     'Missing values roughness_length':df_weather_windpowerlib['roughness_length'].isnull().sum(),
                                    'Observasjoner sim':simulated_data.shape[0],
                                    'Zero values sim':(simulated_data == 0).values.sum()}, index = [Vindpark_navn])
  #Cleaning data
  valid_energy_prod = data_cleaning(valid_energy_prod, Vindpark_navn, Vindpark)

  #Creating a dataframe containing average termperature and and wind speed
  df_avg_climate = pd.DataFrame({'Avg_Temperature':wind['temperature'].mean() ,
                                  'Avg_wind_speed': wind['wind_speed'].mean() }, index = [Vindpark_navn])

  #Identifying and creating a plot of partial downtime near effect lines
  power_curve_wind_farm = windfarm_powercurve(my_turbine, Vindpark)
  new_power_curve = create_80_power_curve(power_curve_wind_farm)
  df_under_pc = find_points_under_powercurve(df_weather_downtime, new_power_curve, Vindpark_navn)

  #Finding downtime near cumulative effect-lines
  df_neigbours = find_neigbour_points(df_cumulative_effect,df_under_pc)

  #Concatinating downtime into a final dataframe called df_downtime
  df_downtime_neigbors = pd.DataFrame(df_neigbours[['wind_speed']])
  df_downtime_windfarm = pd.concat([pd.DataFrame(df_downtime_zero_production['wind_speed']),df_downtime_neigbors])
  df_downtime = pd.DataFrame(index=pd.to_datetime(production_data.index), columns = ['wind_speed'])
  df_downtime = pd.concat([df_downtime, pd.DataFrame(df_downtime_zero_production['wind_speed']),df_downtime_neigbors])
  df_downtime.rename(columns={"wind_speed": Vindpark_navn}, inplace=True)

  #Calculating annual downtime in percent for one wind farm
  df_annual_downtime_percent = ((df_downtime.notnull().astype('int').resample('A').sum()/valid_energy_prod.resample('A').count())*100)

  #Remocing downtime for climate data and production data
  df_weather_windpowerlib_without_downtime = remove_downtime_points(df_weather_windpowerlib_without_zero_downtime, df_neigbours)
  df_valid_prod_without_downtime = remove_downtime_points(df_valid_prod_without_zero_downtime, df_neigbours)

  #df_production_cleaned = valid_energy_prod
  #df_production_cleaned['wind_speed'] = wind['wind_speed'][df_production_cleaned.index]

  #Simulating using data without downtime.
  simulated_data_without_downtime,_ = windpowerlib_simulation(Vindpark,Vindpark_navn, df_weather_windpowerlib_without_downtime)
  #plot_production_powercurve(Vindpark_navn,df_production_cleaned,df_under_pc,df_neigbours,df_cumulative_effect,power_curve_wind_farm,new_power_curve)


  #Running the regression models
  df_results_lifetime, lr_metrics_lf, ran_metrics_lf, arima_metrics_lf, lr_residuals_lf,ran_residuals_lf = methods_regression(simulated_data,valid_energy_prod,Vindpark_navn, 'Month')
  df_results_lr_season,df_results_ran_season, lr_metrics_s, ran_metrics_s = methods_regression(simulated_data,valid_energy_prod,Vindpark_navn, 'Season')
  df_results_lifetime_downtime,lr_metrics_lf_d, ran_metrics_lf_d,arima_metrics_d, lr_residuals_lf_d,ran_residuals_lf_d = methods_regression(simulated_data_without_downtime,df_valid_prod_without_downtime, Vindpark_navn, 'Month')
  df_results_lr_season_downtime,df_results_ran_season_downtime, lr_metrics_s_d, ran_metrics_s_d = methods_regression(simulated_data_without_downtime,df_valid_prod_without_downtime,Vindpark_navn, 'Season')
  df_results_n_years, lr_metrics_y, ran_metrics_y,arima_metrics_y, lr_residuals_y,ran_residuals_y = methods_regression(simulated_data,valid_energy_prod,Vindpark_navn,'Month', 5)
  print(df_results_lr_season)
  return df_results_lifetime ,df_results_lr_season,df_results_ran_season,df_results_lifetime_downtime,df_results_lr_season_downtime,df_results_ran_season_downtime, df_results_n_years,df_data_statistics, df_downtime,df_annual_downtime_percent, df_distance, lr_metrics_lf, ran_metrics_lf, lr_residuals_lf,ran_residuals_lf, lr_metrics_s, ran_metrics_s, arima_metrics_lf,arima_metrics_d,arima_metrics_y, df_avg_climate

df_results_lifetime ,df_results_lr_season,df_results_ran_season,df_results_lifetime_downtime,df_results_lr_season_downtime,df_results_ran_season_downtime, df_results_n_years,df_data_statistics, df_downtime,df_annual_downtime_percent, df_distance, lr_metrics_lf, ran_metrics_lf, lr_residuals_lf,ran_residuals_lf, lr_metrics_s, ran_metrics_s, arima_metrics_lf,arima_metrics_d,arima_metrics_y, df_avg_climate = run_one_windfarm('Sandøy')

In [ ]:
#Running all wind farms and storing results in separate dataframes
def run_multiple_windfarms(chosen_windfarm):
  #Creating all dataframes that are used to store results from the indicidual wind farms
  df_results = pd.DataFrame()
  df_results_downtime = pd.DataFrame()

  df_data_statistics = pd.DataFrame()
  df_results_n_years = pd.DataFrame()
  df_distance = pd.DataFrame()

  df_results_without_downtime = pd.DataFrame()
  df_results_lr_season_without_downtime = pd.DataFrame()
  df_results_ran_season_without_downtime = pd.DataFrame()

  df_results_season_lr = pd.DataFrame()
  df_results_season_ran = pd.DataFrame()

  df_lr_metrics_lf = pd.DataFrame()
  df_ran_metrics_lf = pd.DataFrame()

  df_lr_metrics_s = pd.DataFrame()
  df_ran_metrics_s = pd.DataFrame()

  df_arima_metrics_lf = pd.DataFrame()
  df_arima_metrics_s = pd.DataFrame()
  df_arima_metrics_y = pd.DataFrame()

  df_lr_residuals_lf = pd.DataFrame()
  df_ran_residuals_ran = pd.DataFrame()

  df_downtime = pd.DataFrame()
  df_downtime_percent = pd.DataFrame()

  df_avg_climate = pd.DataFrame()
  #Running on wind farm at the time. All results are stored in different dataframes
  for windfarm in chosen_windfarm:
    print('##################' + windfarm + '###################')
    results_lifetime,results_lr_season,results_ran_season,results_lifetime_without_downtime,results_lr_season_downtime,results_ran_season_downtime, results_n_years,data_statistics,downtime,annual_downtime_percent, distance, lf_metrics_lf, ran_metrics_lf, lr_residuals_lf,ran_residuals_lf, lr_metrics_s, ran_metrics_s,arima_metrics_lf,arima_metrics_d,arima_metrics_y,avg_climate = run_one_windfarm(windfarm)
    print(results_lr_season,results_ran_season)
    df_results = pd.concat([df_results,results_lifetime])
    df_results_season_lr = pd.concat([df_results_season_lr, results_lr_season])
    df_results_season_ran = pd.concat([df_results_season_ran, results_ran_season])

    df_data_statistics = pd.concat([df_data_statistics,data_statistics])
    df_results_without_downtime = pd.concat([df_results_without_downtime,results_lifetime_without_downtime])
    df_results_lr_season_without_downtime = pd.concat([df_results_lr_season_without_downtime, results_lr_season_downtime])
    df_results_ran_season_without_downtime = pd.concat([df_results_ran_season_without_downtime, results_ran_season_downtime])

    df_results_n_years = pd.concat([df_results_n_years, results_n_years])
    df_lr_metrics_lf = pd.concat([df_lr_metrics_lf, lf_metrics_lf])
    df_ran_metrics_lf = pd.concat([df_ran_metrics_lf, ran_metrics_lf])

    df_lr_metrics_s = pd.concat([df_lr_metrics_s, lr_metrics_s])
    df_ran_metrics_s = pd.concat([df_ran_metrics_s, ran_metrics_s])

    df_arima_metrics_lf = pd.concat([df_arima_metrics_lf, arima_metrics_lf])
    df_arima_metrics_s = pd.concat([df_arima_metrics_s, arima_metrics_d])
    df_arima_metrics_y = pd.concat([df_arima_metrics_y, arima_metrics_y])

    df_lr_residuals_lf = pd.concat([df_lr_residuals_lf, lr_residuals_lf])
    df_ran_residuals_ran = pd.concat([df_ran_residuals_ran, ran_residuals_lf])

    df_distance = pd.concat([df_distance,distance])
    df_downtime = pd.concat([df_downtime, downtime])
    df_downtime_percent = pd.concat([df_downtime_percent, annual_downtime_percent], axis = 1)

    df_avg_climate = pd.concat([df_avg_climate,avg_climate])

  return df_results,df_results_n_years,df_results_season_lr,df_results_season_ran,df_results_without_downtime,df_results_lr_season_without_downtime,df_results_ran_season_without_downtime, df_downtime,df_downtime_percent,df_data_statistics, df_distance, df_lr_metrics_lf,df_ran_metrics_lf,df_lr_residuals_lf,df_ran_residuals_ran,df_lr_metrics_s,df_ran_metrics_s, df_arima_metrics_lf,df_arima_metrics_s, df_arima_metrics_y, df_avg_climate
#List containg the chosen wind farms
chosen = ['Egersund','Fakken','Hamnefjell','Hitra','Kjøllefjord','Lista','Raggovidda','Rye Vind','Røyrmyra','Sandøy','Skomakerfjellet','Tellenes','Valsneset','Ytre Vikna','Åsen II']

#Running the function above
results, results_n_years,results_season_lr,results_season_ran,results_without_downtime,results_lr_season_without_downtime,results_ran_season_without_downtime,df_downtime,df_downtime_percent,df_data_statistics, distance, lr_metrics_lf, ran_metrics_lf,lr_residuals_lf,ran_residuals_ran,lr_metrics_s,ran_metrics_s,df_arima_metrics_lf,df_arima_metrics_s, df_arima_metrics_y, df_avg_climate = run_multiple_windfarms(chosen)





# General results

Only selected models are outputted in this section. However, it is possible to retrived results from the following models: \

-results: general results  from all models using all data \
-results_n_years: results from all models using the first 5 years\
-results_season_lr: seasonal results from a linear regresson\
-results_season_ran: seasonal results from a RANSAC regresson\
-results_without_downtime results from all model using data without partial downtime \
-results_lr_season_without_downtime: seasonal results from linear model using data without downtime\
-results_ran_season_without_downtime: seasonal results from RANSAC model using data without downtime

In [ ]:
#Dataframe of the general results
results

Average and weighted average from the models are then calculated.

In [ ]:
df_results_avg, df_weighted = make_weighted_degradation(results)
df_results_avg

#Plotting of degradation rates

In [ ]:
#Making a DataFrame consisting of geografic-labels and age-labels
df_cluster = pd.DataFrame(index = results.index)
df_cluster['Geografic'] = portfolio_data['Geografic_area']
df_cluster['Age_groups'] = portfolio_data['Age_groups']
df_cluster


In [ ]:
#Adding a geografic label to the main result dataframe and weighted result dataframe
results['Geografic'] = df_cluster['Geografic']
df_weighted['Geografic'] = df_cluster['Geografic']

#Plotting the result of the individual linear regression model to the wind farms
linear_chart = alt.Chart(results.reset_index()).mark_bar(opacity = 0.7).encode(
    x= alt.X('Wind_farm:O',title='Wind farm'),
    y= alt.Y('Linear_degradation:Q',title='Yearly degradation [%]'),
    color = 'Geografic:O',
).properties(
    title='Annual degradation using Linear Regression'
)
linear_weighted_chart = alt.Chart(df_weighted.reset_index()).mark_bar(opacity = 0.7).encode(
    x=alt.X('Wind_farm:O',title='Wind farm'),
    y=alt.Y('Weighted_Linear_degradation:Q',title='[Degradation * proportion]'),
    color = alt.Color('Geografic', scale=alt.Scale(domain = ['South_west', 'Mid', 'North'], range=['blue', 'red', 'green']))
).properties(
    title='Weighted annual degradation using Linear Regression'
)

display(alt.hconcat(linear_chart, linear_weighted_chart).configure_axis(
    labelFontSize=15,
    titleFontSize=20))

In [ ]:
#Plotting the result of the individual RANSAC regression model to the wind farms

ransac_chart = alt.Chart(results.reset_index()).mark_bar(opacity = 0.7).encode(
    x= alt.X('Wind_farm:O',title='Wind farm'),
    y= alt.Y('Ransac_degradation:Q',title='Yearly degradation [%]'),
    color = 'Geografic:O'
).properties(
    title='Annual degradation using RANSAC Regression'
)
ransac_weighted_chart = alt.Chart(df_weighted.reset_index()).mark_bar(opacity = 0.7).encode(
    x=alt.X('Wind_farm:O',title='Wind farm'),
    y=alt.Y('Weighted_Ransac_degradation:Q',title='[Degradation * proportion]'),
    color = alt.Color('Geografic', scale=alt.Scale(domain = ['South_west', 'Mid', 'North'], range=['blue', 'red', 'green']))
).properties(
    title='Weighted annual degradation using Ransac Regression'
)

display(alt.hconcat(ransac_chart, ransac_weighted_chart).configure_axis(
    labelFontSize=15,
    titleFontSize=20))

# Histograms showing distributions of degradation rates

Plotting histograms showing the distrubution of estimated degradation rates. One plot contains all three models and the other contains only Linear and RANSAC regression.

In [ ]:

plt.figure()
plt.hist([results['Linear_degradation'],results['Ransac_degradation'],results['Arima degradation']], bins=30, histtype='bar', label = ['Linear', 'Ransac', 'Arima'])
plt.legend(loc='upper left')
plt.xlabel('Yearly Degradation')
plt.ylabel('Numbers of Wind farms [%]')
plt.show()





In [ ]:

plt.figure()
plt.hist([results['Linear_degradation'],results['Ransac_degradation']], bins=30, histtype='bar', label = ['Linear', 'Ransac'])
plt.legend(loc='upper left')
plt.xlabel('Yearly Degradation [%]')
plt.ylabel('Numbers of Wind farms')
plt.show()

# Results without partial downtime

Results without partial downtime using all data.


In [ ]:
results_without_downtime.set_index('Wind_farm', inplace=True)
results_without_downtime

Average and weighted average from the models are then calculated.

In [ ]:
df_results_avg, df_weighted = make_weighted_degradation(results_without_downtime)
df_results_avg

# Results from the first five production years

Outputting the result from the first five years of production.

In [ ]:
results_n_years.set_index('Wind_farm',inplace=True)
results_n_years

Average and weighted average from the models are then calculated.

In [ ]:
df_results_avg, df_weighted = make_weighted_degradation(results_n_years)
df_results_avg

# Results from the regression of seasonal data.



In [ ]:
results_season_lr.set_index('Wind_farm', inplace=True)
results_season_lr

In [ ]:
results_season_ran.set_index('Wind_farm', inplace=True)
results_season_ran

In [ ]:
results_lr_season_without_downtime.set_index('Wind_farm', inplace=True)
results_lr_season_without_downtime

In [ ]:
results_ran_season_without_downtime.set_index('Wind_farm', inplace=True)
results_ran_season_without_downtime

# Final result

In this section, the final result with confidence interval are calculated.

In [ ]:
avg_result_linear = 0.5* df_results_avg['Ransac Reg Mean'] + 0.5 * df_results_avg['Ransac Reg Weighted Avg']
avg_result_linear

In [ ]:

interval = mean_confidence_interval(results_without_downtime['Ransac_degradation'],df_weighted['Weighted_Ransac_degradation'])
print(interval)

# Estimated downtime

In this section he development of downtime are estimated and different plots of downtime are made.

In [ ]:
#All hours that are considered downtime are marked as 1 and the rest of data points are marked 0
newdf = df_downtime.notnull().astype('int')
newdf

In [ ]:
#Finding total annual hours of downtime for each wind farm
_, one_year = first_valid_hour(newdf, 'Sandøy')
df_monthly = newdf[one_year:].resample('A').sum()

In [ ]:
#plotting annual partial downtime for individual wind farms
df_monthly.plot()
plt.xlabel('Years')
plt.ylabel('Numbers of downtime hours')
plt.legend(loc='upper center', bbox_to_anchor=(1.2, 0.95))
plt.show()

In [ ]:
#Estimated annual partial downtime in percent
df_downtime_percent

In [ ]:
#Finding total annual partial downtime for all wind farms
final_avg = pd.DataFrame(df_downtime_percent.sum(axis=1), columns=['Avg_Degrad_percent'])
final_avg['hrs_count'] = df_downtime_percent.count(axis=1)
final_avg['Annual_degrad'] = final_avg['Avg_Degrad_percent']/final_avg['hrs_count']
final_avg.fillna(0, inplace = True)
final_avg

In [ ]:
# Creating a simple linear regression to find the slope of the partial downtime
final_avg['Time'] = np.arange(len(final_avg.index))
X = final_avg.loc[:, ['Time']].values  # features
y = final_avg.loc[:, ['Annual_degrad']].values  # target

lr = LinearRegression().fit(X,y)
print(f'Annual change in degradation: {float(lr.coef_)} pp/y')

In [ ]:
#Plotting annual downtime and regression line
plt.figure()
plt.scatter(final_avg['Time'], final_avg['Annual_degrad'])
plt.plot(final_avg['Time'], float(lr.intercept_)+ float(lr.coef_) * final_avg['Time'], 'b')
plt.xlabel('Years')
plt.ylabel('Degradation [%]')
plt.show()

# Data statistics and diagnostics

In [ ]:
#Data statistics and diagnostic
df_data_statistics

In [ ]:
#Outputting some statistics of data

print('Number of valid observations:', df_data_statistics['Observasjoner valid prod'].sum())
print('Number of zero values:', df_data_statistics['Zero values prod'].sum())
print('Number of production over capacity:', df_data_statistics['Prod over capacity'].sum())

print('Percent zero values:', (df_data_statistics['Zero values prod'].sum()/df_data_statistics['Observasjoner valid prod'].sum())*100)
print('Percent of over capacity values:', (df_data_statistics['Prod over capacity'].sum()/df_data_statistics['Observasjoner valid prod'].sum())*100)
print('Percent of missing values:', (df_data_statistics['Missing values prod'].sum()/df_data_statistics['Observasjoner valid prod'].sum())*100)
print('Percent of negative values:', (df_data_statistics['Negative values prod'].sum()/df_data_statistics['Observasjoner valid prod'].sum())*100)


# Results metrics

This section contain metrics to the different models. Only metrics to the three main models are outputted now. Metrics for these models are estimated and can be retrieved:



lr_metrics_lf:  linear model of all data
ran_metrics_lf:  RANSAC model of all data
lr_metrics_s: linear model of seasonal data
ran_metrics_s: RANSAC model of seasonal data
df_arima_metrics_lf: ARIMA model of all data
df_arima_metrics_s:ARIMA model of seasonal data








In [ ]:
lr_metrics_lf.set_index('Wind Farm', inplace = True)
lr_metrics_lf

In [ ]:
ran_metrics_lf.set_index('Wind Farm', inplace = True)
ran_metrics_lf

# Results statistical tests

Statistical test for the linear model and RANSAC model are outputted.

In [ ]:
lr_residuals_lf

In [ ]:
ran_residuals_ran

In [ ]:
df_old_turbines = results_n_years.loc[portfolio_data['Age_groups'] == 'Old_turbine']
df_new_turbines = results_n_years.loc[portfolio_data['Age_groups'] == 'New_turbine']

df_south_west = results.loc[portfolio_data['Geografic_area'] == 'South_west']
df_mid = results.loc[portfolio_data['Geografic_area'] == 'Mid']
df_north = results.loc[portfolio_data['Geografic_area'] == 'North']

group = [df_old_turbines,df_new_turbines,df_south_west,df_mid,df_north]

In [ ]:
df_sw_avg_climate = df_avg_climate.loc[portfolio_data['Geografic_area'] == 'South_west']
df_sw_avg_climate

In [ ]:
print('Avg_Temperature, south_west norway',df_sw_avg_climate['Avg_Temperature'].mean()-273.15)
print('Avg_wind_speed, south_west norway',df_sw_avg_climate['Avg_wind_speed'].mean())

In [ ]:
df_mid_avg_climate = df_avg_climate.loc[portfolio_data['Geografic_area'] == 'Mid']
df_mid_avg_climate

In [ ]:
print('Avg_Temperature, mid norway',df_mid_avg_climate['Avg_Temperature'].mean()-273.15)
print('Avg_wind_speed, mid norway',df_mid_avg_climate['Avg_wind_speed'].mean())

In [ ]:
df_north_avg_climate = df_avg_climate.loc[portfolio_data['Geografic_area'] == 'North']
df_north_avg_climate

In [ ]:
print('Avg_Temperature, north norway',df_north_avg_climate['Avg_Temperature'].mean()-273.15)
print('Avg_wind_speed, north norway',df_north_avg_climate['Avg_wind_speed'].mean())

# Grouping of wind farms into different cases

Wind farms are grouped into different locations and into newer and older turbines using DataFrames.

In [ ]:
df_old_turbines

In [ ]:
df_new_turbines

In [ ]:
#Plotting the g

for group in [df_old_turbines,df_new_turbines,df_south_west,df_mid,df_north]:
  print(group)

In [ ]:

for location in [df_south_west,df_mid,df_north]:
  df_results_avg, df_weighted = make_weighted_degradation(location)
  print(df_results_avg)

# Comparison cases

In this sections, comparison groups are defined. These are grouped into larger groups by type of model and type of comparison case. The pre-defined groups are then compared using a Mannwhitney U-test. All the test are conducted by iterating over the pairs.

In [ ]:
#Defining different pairs of groups to compare using MannWhitney U-Test

old_new_lr = {'Old': df_old_turbines['Linear_degradation'], 'New': df_new_turbines['Linear_degradation']}
old_new_ran = {'Old': df_old_turbines['Ransac_degradation'], 'New': df_new_turbines['Ransac_degradation']}


u_test_technology_lr = ('Linear Regression', [old_new_lr])
u_test_technology_ran = ('Ransac Regression',[old_new_ran])

#Defining groups based on geographics
south_mid_lr = {'df_south_west':df_south_west['Linear_degradation'], 'df_mid': df_mid['Linear_degradation']}
south_mid_ran = {'df_south_west': df_south_west['Ransac_degradation'], 'df_mid': df_mid['Ransac_degradation']}

south_north_lr = {'df_south_west':df_south_west['Linear_degradation'], 'df_north': df_north['Linear_degradation']}
south_north_ran = {'df_south_west': df_south_west['Ransac_degradation'], 'df_north': df_north['Ransac_degradation']}

mid_north_lr = {'df_mid':df_mid['Linear_degradation'],'df_north': df_north['Linear_degradation']}
mid_north_ran = {'df_mid': df_mid['Ransac_degradation'], 'df_north': df_north['Ransac_degradation']}


u_test_geografic_lr = ('Linear Regression', [south_mid_lr,south_north_lr,mid_north_lr])
u_test_geografic_ran = ('Ransac Regression',[south_mid_ran,south_north_ran,mid_north_ran])

#Defining groups based on downtime and different life span
results_resultsWODowntime_ran = {'results': results['Ransac_degradation'], 'results_without_downtime': results_without_downtime['Ransac_degradation']}
results_resultsWODowntime_lr = {'results': results['Linear_degradation'], 'results_without_downtime': results_without_downtime['Linear_degradation']}

results_results5_years_lr = {'results': results['Linear_degradation'], 'results_5_years': results_n_years['Linear_degradation']}
results_results5_years_ran ={'results': results['Ransac_degradation'], 'results_5_years': results_n_years['Ransac_degradation']}


u_test_results_lr = ('Linear_degradation', [results_resultsWODowntime_lr,results_results5_years_lr ])
u_test_results_ran = ('Ransac_degradation',[results_resultsWODowntime_lr,results_results5_years_lr ])

#Defining groups based on different seasons
q1_q2_ran = {'Q1': results_season_ran['Q1'], 'Q2': results_season_ran['Q2']}
q1_q3_ran = {'Q1': results_season_ran['Q1'], 'Q3': results_season_ran['Q3']}
q1_q4_ran = {'Q1': results_season_ran['Q1'], 'Q4': results_season_ran['Q4']}
q2_q3_ran = {'Q2': results_season_ran['Q2'], 'Q3': results_season_ran['Q3']}
q2_q4_ran = {'Q2': results_season_ran['Q2'], 'Q4': results_season_ran['Q4']}
q3_q4_ran = {'Q3': results_season_ran['Q3'], 'Q4': results_season_ran['Q4']}

u_test_seasons_ran = ('Ransac Regression', [q1_q2_ran,q1_q3_ran,q1_q4_ran,q2_q3_ran,q2_q4_ran,q2_q4_ran,q3_q4_ran])

q1_q2_lr = {'Q1': results_season_lr['Q1'], 'Q2': results_season_lr['Q2']}
q1_q3_lr = {'Q1': results_season_lr['Q1'], 'Q3': results_season_lr['Q3']}
q1_q4_lr = {'Q1': results_season_lr['Q1'], 'Q4': results_season_lr['Q4']}
q2_q3_lr = {'Q2': results_season_lr['Q2'], 'Q3': results_season_lr['Q3']}
q2_q4_lr = {'Q2': results_season_lr['Q2'], 'Q4': results_season_lr['Q4']}
q3_q4_lr = {'Q3': results_season_lr['Q3'], 'Q4': results_season_lr['Q4']}

u_test_seasons_lr = ('Linear Regression', [q1_q2_lr,q1_q3_lr,q1_q4_lr,q2_q3_lr,q2_q4_lr,q3_q4_lr])

q1_q2_lr_d = {'Q1': results_lr_season_without_downtime['Q1'], 'Q2': results_lr_season_without_downtime['Q2']}
q1_q3_lr_d = {'Q1': results_lr_season_without_downtime['Q1'], 'Q3': results_lr_season_without_downtime['Q3']}
q1_q4_lr_d = {'Q1': results_lr_season_without_downtime['Q1'], 'Q4': results_lr_season_without_downtime['Q4']}
q2_q3_lr_d = {'Q2': results_lr_season_without_downtime['Q2'], 'Q3': results_lr_season_without_downtime['Q3']}
q2_q4_lr_d = {'Q2': results_lr_season_without_downtime['Q2'], 'Q4': results_lr_season_without_downtime['Q4']}
q3_q4_lr_d = {'Q3': results_lr_season_without_downtime['Q3'], 'Q4': results_lr_season_without_downtime['Q4']}

u_test_seasons_wdowntime_lr = ('Linear Regression', [q1_q2_lr_d,q1_q3_lr_d,q1_q4_lr_d,q2_q3_lr_d,q2_q4_lr_d,q3_q4_lr_d])

q1_q2_ran_d = {'Q1': results_ran_season_without_downtime['Q1'], 'Q2': results_ran_season_without_downtime['Q2']}
q1_q3_ran_d = {'Q1': results_ran_season_without_downtime['Q1'], 'Q3': results_ran_season_without_downtime['Q3']}
q1_q4_ran_d = {'Q1': results_ran_season_without_downtime['Q1'], 'Q4': results_ran_season_without_downtime['Q4']}
q2_q3_ran_d = {'Q2': results_ran_season_without_downtime['Q2'], 'Q3': results_ran_season_without_downtime['Q3']}
q2_q4_ran_d = {'Q2': results_ran_season_without_downtime['Q2'], 'Q4': results_ran_season_without_downtime['Q4']}
q3_q4_ran_d = {'Q3': results_ran_season_without_downtime['Q3'], 'Q4': results_ran_season_without_downtime['Q4']}

u_test_seasons_wdowntime_ran = ('Linear Regression', [q1_q2_ran_d,q1_q3_ran_d,q1_q4_ran_d,q2_q3_ran_d,q2_q4_ran_d,q3_q4_ran_d])

u_test_groups = [u_test_technology_lr,u_test_technology_ran,u_test_geografic_lr,u_test_geografic_ran,u_test_results_lr, u_test_results_ran,u_test_seasons_lr,u_test_seasons_ran,u_test_seasons_wdowntime_lr,u_test_seasons_wdowntime_ran]


In [ ]:
#Running all testing pair using the for-loop.
for test_group in u_test_groups:
  print(run_u_test_case(test_group))

